In [1]:
import pulp

In [2]:
products = ['wrenches', 'pliers']
price = [130, 100]

capacity_ub = [15, 16]
steel = [1.5, 1]
molding = [1, 1]
assembly = [0.3, 0.5]

steelprice = 58
capsteel = 27
capmolding = 21
LB = [0,0]

In [3]:
scenarios = [0, 1, 2, 3]
pscenario = [0.25, 0.25, 0.25, 0.25]
wrenchearnings = [160, 160, 90, 90]
plierearnings = [100, 100, 100, 100]
capassembly = [8, 10, 8, 10]

In [4]:
production = [(j,i) for j in scenarios for i in products]
pricescenario = [[wrenchearnings[j], plierearnings[j]] for j in scenarios]
priceitems = [item for sublist in pricescenario for item in sublist]

In [5]:
print(production)
print(pricescenario)
print(priceitems)

[(0, 'wrenches'), (0, 'pliers'), (1, 'wrenches'), (1, 'pliers'), (2, 'wrenches'), (2, 'pliers'), (3, 'wrenches'), (3, 'pliers')]
[[160, 100], [160, 100], [90, 100], [90, 100]]
[160, 100, 160, 100, 90, 100, 90, 100]


In [6]:
price_dict = dict(zip(production, priceitems))
capacity_dict = dict(zip(products, capacity_ub*4))
steel_dict = dict(zip(products, steel))
molding_dict = dict(zip(products, molding))
assembly_dict = dict(zip(products, assembly))

In [7]:
print(price_dict)
print('\n')

print(capacity_dict)
print(steel_dict)
print(molding_dict)
print(assembly_dict)

{(0, 'wrenches'): 160, (0, 'pliers'): 100, (1, 'wrenches'): 160, (1, 'pliers'): 100, (2, 'wrenches'): 90, (2, 'pliers'): 100, (3, 'wrenches'): 90, (3, 'pliers'): 100}


{'wrenches': 15, 'pliers': 16}
{'wrenches': 1.5, 'pliers': 1}
{'wrenches': 1, 'pliers': 1}
{'wrenches': 0.3, 'pliers': 0.5}


In [8]:
production_vars = pulp.LpVariable.dicts("production", (scenarios, products), lowBound=0, cat='Continuous')
steelpurchase = pulp.LpVariable("steelpurchase", lowBound=0, cat='Continuous')

In [9]:
production_vars

{0: {'wrenches': production_0_wrenches, 'pliers': production_0_pliers},
 1: {'wrenches': production_1_wrenches, 'pliers': production_1_pliers},
 2: {'wrenches': production_2_wrenches, 'pliers': production_2_pliers},
 3: {'wrenches': production_3_wrenches, 'pliers': production_3_pliers}}

In [10]:
steelpurchase

steelpurchase

In [11]:
gemstoneprob = pulp.LpProblem("The Gemstone Tool Problem",pulp.LpMaximize)

In [12]:
print(price_dict)
print('\n')
print(pscenario)
print('\n')

print(production)

{(0, 'wrenches'): 160, (0, 'pliers'): 100, (1, 'wrenches'): 160, (1, 'pliers'): 100, (2, 'wrenches'): 90, (2, 'pliers'): 100, (3, 'wrenches'): 90, (3, 'pliers'): 100}


[0.25, 0.25, 0.25, 0.25]


[(0, 'wrenches'), (0, 'pliers'), (1, 'wrenches'), (1, 'pliers'), (2, 'wrenches'), (2, 'pliers'), (3, 'wrenches'), (3, 'pliers')]


In [13]:
production_vars

{0: {'wrenches': production_0_wrenches, 'pliers': production_0_pliers},
 1: {'wrenches': production_1_wrenches, 'pliers': production_1_pliers},
 2: {'wrenches': production_2_wrenches, 'pliers': production_2_pliers},
 3: {'wrenches': production_3_wrenches, 'pliers': production_3_pliers}}

In [14]:
for i in  production:
    print(i)

(0, 'wrenches')
(0, 'pliers')
(1, 'wrenches')
(1, 'pliers')
(2, 'wrenches')
(2, 'pliers')
(3, 'wrenches')
(3, 'pliers')


In [21]:
for i in  production:
    production_vars(i)

TypeError: 'dict' object is not callable

In [15]:
gemstoneprob += pulp.lpSum([ pscenario[j] * (price_dict[(j,i)] * production_vars[j][i]) for (j,i) in  production ] - steelpurchase * steelprice)

In [16]:
for j in scenarios:
    gemstoneprob += pulp.lpSum([steel_dict[i] * production_vars[j][i] \
                                for i in products]) - \
                                steelpurchase <= 0, ("Steel capacity" + str(j))
    gemstoneprob += pulp.lpSum([molding_dict[i] * production_vars[j][i] \
                                for i in products]) <= capmolding, \
                                ("molding capacity" +str(j))
    gemstoneprob += pulp.lpSum([assembly_dict[i] * production_vars[j][i] \
                                for i in products]) <= capassembly[j], \
                                ("assembly capacity" +str(j))
    for i in products:
        gemstoneprob += production_vars[j][i] <= capacity_dict[i], \
                                            ("capacity " + str(i) + str(j))


In [17]:
# Print problem
print(gemstoneprob)

The Gemstone Tool Problem:
MAXIMIZE
25.0*production_0_pliers + 40.0*production_0_wrenches + 25.0*production_1_pliers + 40.0*production_1_wrenches + 25.0*production_2_pliers + 22.5*production_2_wrenches + 25.0*production_3_pliers + 22.5*production_3_wrenches + -58*steelpurchase + 0.0
SUBJECT TO
Steel_capacity0: production_0_pliers + 1.5 production_0_wrenches
 - steelpurchase <= 0

molding_capacity0: production_0_pliers + production_0_wrenches <= 21

assembly_capacity0: 0.5 production_0_pliers + 0.3 production_0_wrenches <= 8

capacity_wrenches0: production_0_wrenches <= 15

capacity_pliers0: production_0_pliers <= 16

Steel_capacity1: production_1_pliers + 1.5 production_1_wrenches
 - steelpurchase <= 0

molding_capacity1: production_1_pliers + production_1_wrenches <= 21

assembly_capacity1: 0.5 production_1_pliers + 0.3 production_1_wrenches <= 10

capacity_wrenches1: production_1_wrenches <= 15

capacity_pliers1: production_1_pliers <= 16

Steel_capacity2: production_2_pliers + 1.5 p

In [18]:
# The problem data is written to an .lp file
gemstoneprob.writeLP("gemstoneprob.lp")
# The problem is solved using PuLP's choice of Solver
gemstoneprob.solve()
# The status of the solution is printed to the screen
print("Status:", pulp.LpStatus[gemstoneprob.status])

Status: Optimal


In [19]:
# Each of the variables is printed with it's resolved optimum value
for v in gemstoneprob.variables():
    print(v.name, "=", v.varValue)
production = [v.varValue for v in gemstoneprob.variables()]
production

# The optimised objective function value is printed to the console
print("Total price = ", pulp.value(gemstoneprob.objective))

production_0_pliers = 4.75
production_0_wrenches = 15.0
production_1_pliers = 4.75
production_1_wrenches = 15.0
production_2_pliers = 8.5
production_2_wrenches = 12.5
production_3_pliers = 16.0
production_3_wrenches = 5.0
steelpurchase = 27.25
Total price =  863.25
